# Random Forest Model

In [37]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error

In [6]:
user = pd.read_csv('users.csv')
user

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN
...,...,...,...
278853,278854,"portland, oregon, usa",NaN
278854,278855,"tacoma, washington, united kingdom",50.0
278855,278856,"brampton, ontario, canada",NaN
278856,278857,"knoxville, tennessee, usa",NaN


In [7]:
rating = pd.read_csv('ratings.csv')
rating

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6
...,...,...,...
1149775,276704,1563526298,9
1149776,276706,0679447156,0
1149777,276709,0515107662,10
1149778,276721,0590442449,10


In [8]:
p = pd.merge(user, rating.rename(columns={'User-ID':'User-ID'}), on='User-ID',  how='left')



In [9]:
book = pd.read_csv('books.csv')
df = pd.merge(p, book.rename(columns={'ISBN':'ISBN'}), on='ISBN',  how='left')
df = df.drop(columns=['Image-URL-S', 'Image-URL-M', 'Image-URL-L'])
df

/var/folders/sq/fjrsnyh973z09v3bvdz5ffc40000gn/T/ipykernel_16815/1224839744.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  book = pd.read_csv('books.csv')


,User-ID,Location,Age,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher
0,1,"nyc, new york, usa",NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,"stockton, california, usa",18.0,0195153448,0.0,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press
2,3,"moscow, yukon territory, russia",NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,"porto, v.n.gaia, portugal",17.0,NaN,NaN,NaN,NaN,NaN,NaN
4,5,"farnborough, hants, united kingdom",NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
1323350,278854,"portland, oregon, usa",NaN,0553579606,8.0,Ashes to Ashes,TAMI HOAG,2000,Bantam
1323351,278855,"tacoma, washington, united kingdom",50.0,NaN,NaN,NaN,NaN,NaN,NaN
1323352,278856,"brampton, ontario, canada",NaN,NaN,NaN,NaN,NaN,NaN,NaN
1323353,278857,"knoxville, tennessee, usa",NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
df1 = df[['Location','Book-Title','Book-Author','Publisher']]
str_cols = df1.columns[df1.dtypes.eq('object')]

clfs = {c:LabelEncoder() for c in str_cols}
for col, clf in clfs.items():
    df1[col] = clfs[col].fit_transform(df1[col])
df1
        

/var/folders/sq/fjrsnyh973z09v3bvdz5ffc40000gn/T/ipykernel_16815/2822056783.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1[col] = clfs[col].fit_transform(df1[col])


,Location,Book-Title,Book-Author,Publisher
0,36772,241071,101588,16729
1,48830,36197,64911,10924
2,33483,241071,101588,16729
3,40562,241071,101588,16729
4,16902,241071,101588,16729
...,...,...,...,...
1323350,40494,16861,93340,1414
1323351,49760,241071,101588,16729
1323352,6984,241071,101588,16729
1323353,25771,241071,101588,16729


In [32]:
df2 = df[['Age','Book-Rating']]
df3 = df1.join(df2)

df3 = df3[~df.isin([np.nan, np.inf, -np.inf]).any(1)]
#r = [x for x in df3['ISBN'] if str(x).endswith(('x','X'))]
#df4 = df3[~df3.ISBN.isin(r)]
#df4.info()
df3

,Location,Book-Title,Book-Author,Publisher,Age,Book-Rating
1,48830,36197,64911,10924,18.0,0.0
28,1212,129009,14081,12962,26.0,0.0
48,55230,142849,627,12398,14.0,7.0
49,27169,34784,90404,1714,19.0,0.0
71,33195,210121,23200,5370,24.0,0.0
...,...,...,...,...,...,...
1323337,13344,108033,35699,14743,33.0,8.0
1323338,13344,135187,91747,12028,33.0,0.0
1323339,13344,43302,62810,673,33.0,10.0
1323340,13344,83649,14892,13426,33.0,7.0


In [39]:
X = df3.iloc[:, :5]
y = df3['Book-Rating']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [40]:
rf = RandomForestRegressor(n_estimators = 50,  max_depth = 5).fit(X_train, y_train)


In [41]:
prediction = rf.predict(X_test)
mse = mean_squared_error(y_test, prediction)
rmse = mse**.5
print(mse)
print(rmse)

14.660631830999439
3.828920452425127
